<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_0618/blob/main/tibame_0709_ptt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
403 Forbidden
1. IP ban: 換ip
2. 模仿的不像: 少了必要的Headers

```



In [7]:
import urllib.request as req
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/M.1751962851.A.45A.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/1")
resp = req.urlopen(r)
content = resp.read()
html = bs.BeautifulSoup(content)
# print(html)

metas = html.find_all("span", {"class":"article-meta-value"})
author = metas[0]
board = metas[1]
title = metas[2]
post_time = metas[3]
push_datas = html.find_all("div", {"class":"push"})


author_text = author.text.strip()
board_text = board.text.strip()
title_text = title.text.strip()
post_time_text = post_time.text.strip()

pushes = []
for p in push_datas:
    push_meta = p.find_all("span")
    # extract
    push_tag = push_meta[0]
    push_uid = push_meta[1]
    push_content = push_meta[2]
    push_ipdatetime = push_meta[3]
    trans = {"推":1, "噓":-1, "→":0}
    # text
    push_data = {
        "push_tag":trans[push_tag.text.strip()],
        "push_uid":push_uid.text.strip(),
        "push_content":push_content.text.strip().replace(": ", ""),
        "push_ipdatetime":push_ipdatetime.text.strip()
    }
    pushes.append(push_data)


row = {
    "author":author_text,
    "board":board_text,
    "title":title_text,
    "post_time":post_time_text,
    "pushes":pushes,
}
print(row)

{'author': 'a0802459 (王月可可)', 'board': 'Beauty', 'title': '[正妹] 短髮李珠珢', 'post_time': 'Tue Jul  8 16:20:49 2025', 'pushes': [{'push_tag': 1, 'push_uid': 'hyperfrog', 'push_content': '身材很好 但短髮有點莎味', 'push_ipdatetime': '42.72.86.237 07/08 16:23'}, {'push_tag': -1, 'push_uid': 'kenslc199', 'push_content': '越看越像izsasa', 'push_ipdatetime': '106.64.97.99 07/08 16:26'}, {'push_tag': 1, 'push_uid': 'haircc', 'push_content': '鷰尾', 'push_ipdatetime': '116.59.85.235 07/08 16:29'}, {'push_tag': 1, 'push_uid': 'njnjy', 'push_content': '珠珠寶貝', 'push_ipdatetime': '36.233.28.49 07/08 16:33'}, {'push_tag': 1, 'push_uid': 'loloman', 'push_content': '真的很漂亮', 'push_ipdatetime': '111.71.215.198 07/08 16:35'}, {'push_tag': 1, 'push_uid': 'thewtf', 'push_content': '我老婆', 'push_ipdatetime': '49.217.198.113 07/08 16:38'}, {'push_tag': 1, 'push_uid': 'Krishna', 'push_content': '害我覺得莎莎愈看愈可愛', 'push_ipdatetime': '49.217.135.204 07/08 16:45'}, {'push_tag': 1, 'push_uid': 'CavendishJr', 'push_content': '100%的愛莉莎莎', 